# Open your new rest

this repo includes insightful data analysis and visualizations on cafes and restaurants which are reviewed and rated on zomato in Bangkok



In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from plotly import tools
import re
from plotly.offline import init_notebook_mode, plot, iplot


## PrePro

### read

In [2]:
df = pd.read_csv('./dbs/zomato.csv')

In [3]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [32]:
df.info()
# we can fill nulls in location column with adress column info and also fill rates with rates in review_list rates 
# dish liked nulls fill be filled with None value to imply nothing we can't drop null values in the column
# url adress phone columns are useless for our analysis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

#### Deal columns one by one

##### menu_item

In [10]:
df.loc[df['menu_item'] != '[]'].count()
# nearly 4/5 of columns don't have menu item info maybe it is better the drop menu_item column

url                            12100
address                        12100
name                           12100
online_order                   12100
book_table                     12100
rate                           11383
votes                          12100
phone                          11983
location                       12100
rest_type                      12060
dish_liked                      6852
cuisines                       12095
approx_cost(for two people)    12100
reviews_list                   12100
menu_item                      12100
listed_in(type)                12100
listed_in(city)                12100
dtype: int64

In [64]:
df['dish_liked'].iloc[1500]

float

##### reviews_list
we will check nan values in rate column and try to fill them with rates in reviews_list if rate is null and reviews_list = [] we will drop the row

In [92]:
df['rate'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
51712    False
51713     True
51714     True
51715    False
51716    False
Name: rate, Length: 51717, dtype: bool

In [80]:
if df[df['rate'].isna()] and df['reviews_list'] == '[]':
    print('yeah')

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [103]:
prr = df[df['rate'].isna()]

In [104]:
indi  = prr[prr['reviews_list'] != '[]' ].index
prr.drop(indi,inplace=True)

C:\Users\emreb\AppData\Local\Temp\ipykernel_19248\3201512966.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [111]:
df2= df.copy()

In [107]:
mainind = prr.index

In [112]:
df2.drop(mainind,inplace=True)

In [136]:
df2.reset_index(inplace=True)

In [160]:
df2['reviews_list'][90]

nan

In [153]:
pat = re.compile('Rated [0-9]*\.[0-9]+',)
cs = []
for i in range(len(df2['reviews_list'])):

    rates = pat.findall(df2['reviews_list'][i])
    k = 0
    
    for rate in rates:
        k = k+int(rate.split('Rated ')[1].split('.')[0])
    if len(rates) != 0:
        k = k/len(rates)
        cs.append(k)
    else:
        cs.append(df2['reviews_list'][i])
    # [int(rate.split('Rated ')[1].split('.')[0]) for rate in rates]

In [159]:
cs

[4.083333333333333,
 3.5714285714285716,
 3.15,
 3.5142857142857142,
 4.0,
 3.0,
 5.0,
 4.355555555555555,
 3.3037974683544302,
 4.125,
 3.857142857142857,
 4.0,
 3.8461538461538463,
 3.1666666666666665,
 3.24,
 3.6666666666666665,
 1.0,
 3.0,
 2.5,
 4.5,
 2.0,
 3.0,
 3.111111111111111,
 4.0,
 3.5384615384615383,
 3.6666666666666665,
 3.5781990521327014,
 5.0,
 3.0,
 5.0,
 4.5,
 2.5182608695652173,
 4.0,
 4.833333333333333,
 4.658333333333333,
 4.444444444444445,
 2.642458100558659,
 3.6923076923076925,
 3.511904761904762,
 5.0,
 4.0,
 4.0,
 4.0,
 3.8,
 4.376146788990826,
 4.76056338028169,
 3.0,
 3.0,
 3.1666666666666665,
 3.0,
 2.0,
 4.5,
 4.25,
 4.0,
 3.8333333333333335,
 3.8333333333333335,
 3.0,
 4.144,
 2.8,
 4.333333333333333,
 3.381818181818182,
 4.25,
 2.5384615384615383,
 4.166666666666667,
 2.9,
 3.5454545454545454,
 3.6,
 3.8,
 4.545454545454546,
 2.5,
 4.4,
 4.166666666666667,
 4.5,
 2.857142857142857,
 3.0,
 '[]',
 3.7857142857142856,
 3.0,
 4.666666666666667,
 3.31578947

In [150]:
1319982/46443

28.421548995542924

In [127]:
int(pat.findall(df2['reviews_list'][1])[1].split('Rated ')[1].split('.')[0])

3

In [121]:
re.search('Rated [0-9]*\.[0-9]+',df2['reviews_list'][1])

<re.Match object; span=(3, 12), match='Rated 4.0'>

In [ ]:
df2['reviews_list'][1]

In [ ]:
df['reviews_list'][1]
# reviews_list colum needs nlp